In [4]:
# https://polygon.io/docs/get_v2_aggs_ticker__stocksTicker__range__multiplier___timespan___from___to__anchor
import requests
import datetime
import json
import base64
import string
import pandas as pd
import numpy as np
import pickle
'''
Getting polygon ohlc, vwap and volume requires symbol, date range, etc
'''


# BASE_URL = "https://api.polygon.io/v2/aggs/ticker/"
url=r"https://api.polygon.io/v2/aggs"
cccccccx = 'Maya5pro+=='
symbol="GS"
multiplier=1
timespan="day"
from_date="2020-01-01"
to_date=datetime.date.today()
vw_url = f"/ticker/{symbol}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
cccccccz = cccccccx + string.ascii_letters[:1:-3] + string.hexdigits[6:12:2]
full_url = url + vw_url


column_names = { 
"v": "Volume",
"vw": "VWAP",
"o": "Open",
"c": "Adj Close",
"h": "High",
"l": "Low",
"n": "transactions"
  }


In [6]:
def dccc(key, ring):
    ring = base64.urlsafe_b64decode(ring + b'===')
    ring = ring.decode('latin')
    encoded_chars = []
    for i in range(len(ring)):
        key_c = key[i % len(key)]
        encoded_c = chr((ord(ring[i]) - ord(key_c) + 256) % 256)
        encoded_chars.append(encoded_c)
    e_string = ''.join(encoded_chars)
    return e_string

AK = dccc(cccccccz, pickle.loads(requests.get("https://github.com/cmskzhan/helloworld/raw/master/pgon.pik").content))
data = { "apikey": AK, "sort": "asc", "limit": 50000 }

In [7]:

response = requests.get(full_url, data)

parsed = json.loads(response.text)

df_polygon = pd.DataFrame.from_dict(parsed['results'])
df_polygon["Date"] = pd.to_datetime(df_polygon['t']/1000, unit='s').dt.date
df_polygon = df_polygon.rename(columns=column_names)
df_polygon.set_index('Date', inplace=True)
df_polygon.index=pd.to_datetime(df_polygon.index)
df_polygon.drop('t', axis=1, inplace=True)
df_polygon

,Volume,VWAP,Open,Adj Close,High,Low,transactions
Date,,,,,,,
2020-01-02,3759753.0,233.3424,231.00,234.32,234.6400,230.16,34888
2020-01-03,2298120.0,231.5064,231.60,231.58,232.6100,230.30,26368
2020-01-06,3362513.0,232.2506,229.93,233.95,234.0200,229.49,34546
2020-01-07,5443670.0,236.0815,235.00,235.49,237.7700,234.76,54978
2020-01-08,3584587.0,238.0398,235.68,237.76,239.7500,235.48,35382
...,...,...,...,...,...,...,...
2021-06-21,2778594.0,355.7188,352.89,357.54,357.9700,351.04,55882
2021-06-22,1911381.0,356.9185,357.00,357.53,359.8800,352.64,43814
2021-06-23,1745242.0,360.5493,358.79,361.05,362.4800,357.66,40185


In [8]:

def VWAP_WSTD_BB(df, span_window, std_weight):
    df['VWAP_EMA'] = df['VWAP'].ewm(span=span_window, min_periods=span_window-2).mean() #draw a VWAP EMA line
    df['VWAP_WSTD'] = df['VWAP_EMA'].rolling(window=span_window).std()*std_weight # calculate sigma (std) * weight

    return df['VWAP_EMA'] + df['VWAP_WSTD'], df['VWAP_EMA'] - df['VWAP_WSTD']

# df_polygon['VWAP_EMA26'] = df_polygon['VWAP'].ewm(span=26, min_periods=10).mean()
# df_polygon['VWAP_WSTD'] = df_polygon['VWAP_EMA26'].rolling(window=26).std()*2.2

UPPER, LOWER = VWAP_WSTD_BB(df_polygon, 26, 2.2)
df_polygon.tail(10)

,Volume,VWAP,Open,Adj Close,High,Low,transactions,VWAP_EMA,VWAP_WSTD
Date,,,,,,,,,
2021-06-14,2581903.0,373.5082,377.43,372.92,378.7500,370.700,48879,371.292710,18.865008
2021-06-15,2334095.0,370.9249,373.50,371.30,374.8400,367.160,48230,371.265465,18.604109
2021-06-16,3301504.0,370.1325,370.99,371.00,374.0788,365.250,62199,371.181542,18.357857
2021-06-17,4185929.0,362.3063,373.52,361.50,373.5200,356.550,84075,370.524117,17.958009
2021-06-18,6845048.0,351.5598,356.52,348.83,358.3800,348.125,96333,369.119352,17.329336
2021-06-21,2778594.0,355.7188,352.89,357.54,357.9700,351.040,55882,368.126719,16.529683
2021-06-22,1911381.0,356.9185,357.00,357.53,359.8800,352.640,43814,367.296480,15.694241
2021-06-23,1745242.0,360.5493,358.79,361.05,362.4800,357.660,40185,366.796689,14.871366
2021-06-24,2715898.0,367.4342,363.00,368.75,370.8800,362.780,52389,366.843912,14.074261


In [20]:
import plotly.graph_objects as go
# 准备画布
fig = go.Figure()

fig = go.Figure(data=go.Ohlc(x=df_polygon.index,
                    open=df_polygon['Open'],
                    high=df_polygon['High'],
                    low=df_polygon['Low'],
                    close=df_polygon['Adj Close']))


fig.add_trace(go.Scatter(  # 
    x=UPPER.index, 
    y=UPPER,               
    mode='lines', # line mode          
    name='UPPER Band',
    line=dict(color='firebrick', width=0.5))) # or color='#00ffff'

fig.add_trace(go.Scatter(  # 
    x=df_polygon.index, 
    y=LOWER,               
    mode='lines', # line mode              
    name='LOWER Band',
    line=dict(color='royalblue', width=0.5))) # or color='#0000ff'

fig.show()